# Importing datasets
* Converts data obtained form the AniList API into a shared format
* Note that loading and saving the datasets may take several minutes

In [ ]:
import datetime
import glob
import logging
import os

import pandas as pd

In [ ]:
media = ""

In [ ]:
source_dir = "../../data/anilist/"

In [ ]:
outdir = "../../data/raw_data"
if not os.path.exists(outdir):
    os.mkdir(outdir)

In [ ]:
# logging
logger = logging.getLogger("AniList")
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    "%(name)s:%(levelname)s:%(asctime)s: %(message)s", datefmt="%Y-%m-%d %H:%M:%S"
)
for stream in [
    logging.StreamHandler(),
]:
    stream.setFormatter(formatter)
    logger.addHandler(stream)

In [ ]:
%run ImportAnilistUserLists.ipynb

In [ ]:
if os.path.exists(os.path.join(outdir, f"user_{media}_list.anilist.csv")):
    os.remove(os.path.join(outdir, f"user_{media}_list.anilist.csv"))

In [ ]:
for file in glob.glob(os.path.join(source_dir, f"user_{media}_facts/user_{media}_list.*.csv")):
    verify_user_media_list_consistency(
        os.path.join(source_dir, file)
    )

In [ ]:
for file in glob.glob(os.path.join(source_dir, f"user_{media}_facts/user_{media}_list.*.csv")):
    process_user_media_lists(
        file,
        os.path.join(outdir, f"user_{media}_list.anilist.csv"),
    )